# Middleware: Human In The Loop
<img src="./assets/LC_HITL.png" width="300">



## Setup

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

In [ ]:
# Imports
import nest_asyncio
from typing import Annotated, List, TypedDict, Literal, Optional
from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

nest_asyncio.apply()

# Load database (ensure 'Chinook.db' is in current directory)
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

# State includes pending tool call for interrupt simulation
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    pending_tool_call: Optional[dict]

# Tool: execute SQL
@tool(description="Execute a read-only SQLite SELECT query.")
def execute_sql(query: str) -> str:
    try:
        if not query.strip().upper().startswith("SELECT"):
            return "Error: Only SELECT allowed."
        clean = query.strip().replace('```sql', '').replace('```', '').rstrip(';') + ';'
        return str(db.run(clean))
    except Exception as e:
        return f"Error: {e}"

# LLM
llm = ChatOllama(model="qwen2.5-coder:7b", temperature=0.8)

# System prompt with real schema
SYSTEM_PROMPT = f"""You are a careful SQL analyst for the Chinook music store.

Schema:
{db.get_table_info()}

Rules:
- Use correct table and column names (e.g., Employee.FirstName, not 'name').
- Output only one SELECT query when needed.
- Do not guess column names.
- If you cannot answer, say so clearly.
"""

# Node: Call LLM
def call_model(state: AgentState):
    sys_msg = SystemMessage(content=SYSTEM_PROMPT)
    response = llm.invoke([sys_msg] + state["messages"])
    
    # Fallback: extract SQL if no tool_calls
    if not getattr(response, 'tool_calls', None):
        import re
        match = re.search(r'(SELECT\s+.*?)(?:;|\s*$)', response.content, re.IGNORECASE | re.DOTALL)
        if match:
            response.tool_calls = [{
                "name": "execute_sql",
                "args": {"query": match.group(1).strip() + ";"},
                "id": "manual"
            }]
    
    return {"messages": [response]}

# Node: Capture pending tool call (for approval)
def request_approval(state: AgentState):
    last_ai = state["messages"][-1]
    tool_call = last_ai.tool_calls[0] if last_ai.tool_calls else None
    return {"pending_tool_call": tool_call}

# Router after LLM
def route_after_llm(state: AgentState) -> Literal["request_approval", "__end__"]:
    last = state["messages"][-1]
    if getattr(last, 'tool_calls', None):
        return "request_approval"
    return "__end__"

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("request_approval", request_approval)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    route_after_llm,
    {"request_approval": "request_approval", "__end__": END}
)

# Compile app
app = workflow.compile(checkpointer=MemorySaver())

# Simulated human-in-the-loop runner
async def run_with_human_approval(question: str, thread_id: str, approve: bool = True):
    config = {"configurable": {"thread_id": thread_id}}
    inputs = {"messages": [HumanMessage(content=question)]}
    
    # Run graph until it stops (at request_approval or end)
    result = await app.ainvoke(inputs, config)
    
    last_msg = result["messages"][-1]
    
    # Check if a tool call was made (needs approval)
    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
        tool_call = last_msg.tool_calls[0]
        query = tool_call["args"]["query"]
        
        print(f"\n{'='*60}")
        print(f"🛑 INTERRUPT: Query requires human approval:")
        print(f"   {query}")
        print(f"   Decision: {'APPROVED' if approve else 'REJECTED'}")
        print('='*60)
        
        # Simulate human decision
        if approve:
            tool_output = execute_sql(query)
        else:
            tool_output = "Query rejected by human reviewer."
        
        # Inject tool response back into the conversation
        tool_message = ToolMessage(
            content=tool_output,
            tool_call_id=tool_call["id"],
            name=tool_call["name"]
        )
        
        # Continue processing with tool result
        final_state = await app.ainvoke({"messages": [tool_message]}, config)
        
        # Generate final natural-language answer
        sys_msg = SystemMessage(content=SYSTEM_PROMPT)
        final_response = llm.invoke([sys_msg] + final_state["messages"])
        print(f"\n[AI] {final_response.content}")
    
    else:
        # No tool call needed
        print(f"\n[AI] {last_msg.content}")

# 🔴 FIRST: Test with REJECTED (False)
await run_with_human_approval("What are the names of all the employees?", "run_reject", approve=False)

# 🟢 THEN: Test with APPROVED (True)
await run_with_human_approval("What are the names of all the employees?", "run_approve", approve=True)